<a href="https://colab.research.google.com/github/ai-for-dld/ai_for_dld_udemy/blob/main/colab/github_copilot_0203_installing_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing GHDL, Icarus, and Yosys in Google Colab

This notebook will guide you through installing the essential open-source tools for digital hardware design and verification:

- **GHDL**: A VHDL simulator for simulating VHDL designs and testbenches.
- **Icarus Verilog (iverilog)**: A Verilog simulator for simulating Verilog designs and testbenches.
- **Yosys**: A synthesis tool for Verilog (and limited VHDL via plugins), transforming HDL designs into gate-level netlists for implementation on FPGAs or ASICs.

Together, these tools allow you to explore both simulation and synthesis flows for digital logic using both VHDL and Verilog languages.

In [1]:
%%bash
apt-get update -y
apt-get install -y ghdl-llvm iverilog yosys

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,461 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [5,139 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/un

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
%%bash
echo "GHDL version:"
ghdl --version
echo "\nIcarus Verilog version:"
iverilog -V
echo "\nYosys version:"
yosys -V

GHDL version:
GHDL 1.0.0 (Ubuntu 1.0.0+dfsg-6) [Dunoon edition]
 Compiled with GNAT Version: 10.3.1 20211117
 llvm code generator
Written by Tristan Gingold.

Copyright (C) 2003 - 2021 Tristan Gingold.
GHDL is free software, covered by the GNU General Public License.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
\nIcarus Verilog version:
Icarus Verilog version 11.0 (stable) ()

Copyright 1998-2020 Stephen Williams

  This program is free software; you can redistribute it and/or modify
  it under the terms of the GNU General Public License as published by
  the Free Software Foundation; either version 2 of the License, or
  (at your option) any later version.

  This program is distributed in the hope that it will be useful,
  but WITHOUT ANY WARRANTY; without even the implied warranty of
  MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
  GNU General Public License for more details.

  You should have received a copy of the GNU G

We will demonstrate both **Verilog** and **VHDL** in this notebook, showcasing a simple 2-input AND gate and its simulation using the installed tools.

In [3]:
# Write Verilog AND gate and testbench
with open('and_gate.v', 'w') as f:
    f.write('''
// and_gate.v
module and_gate(
    input a,
    input b,
    output out
);
assign out = a & b;
endmodule
''')

with open('and_gate_tb.v', 'w') as f:
    f.write('''
// and_gate_tb.v
`timescale 1ns / 1ps
module and_gate_tb;
  reg a;
  reg b;
  wire out;
  and_gate uut (
    .a(a),
    .b(b),
    .out(out)
  );
  initial begin
    a = 0; b = 0;
    $dumpfile("and_gate.vcd");
    $dumpvars(0, and_gate_tb);
    #10 a = 0; b = 1;
    #10 a = 1; b = 0;
    #10 a = 1; b = 1;
    #10 a = 0; b = 0;
    #10 $finish;
  end
endmodule
''')

# Write VHDL AND gate and testbench
with open('and_gate.vhd', 'w') as f:
    f.write('''
-- and_gate.vhd
library ieee;
use ieee.std_logic_1164.all;
entity and_gate is
    port (
        a : in std_logic;
        b : in std_logic;
        out_port : out std_logic
    );
end entity and_gate;
architecture behavioral of and_gate is
begin
    out_port <= a and b;
end architecture behavioral;
''')

with open('and_gate_tb.vhd', 'w') as f:
    f.write('''
-- and_gate_tb.vhd
library ieee;
use ieee.std_logic_1164.all;
entity and_gate_tb is end entity and_gate_tb;
architecture test of and_gate_tb is
    component and_gate
        port (
            a : in std_logic;
            b : in std_logic;
            out_port : out std_logic
        );
    end component;
    signal a_tb : std_logic := '0';
    signal b_tb : std_logic := '0';
    signal out_tb : std_logic;
begin
    uut: and_gate port map(
        a => a_tb,
        b => b_tb,
        out_port => out_tb
    );
    process
    begin
        wait for 10 ns;
        a_tb <= '0'; b_tb <= '1';
        wait for 10 ns;
        a_tb <= '1'; b_tb <= '0';
        wait for 10 ns;
        a_tb <= '1'; b_tb <= '1';
        wait for 10 ns;
        a_tb <= '0'; b_tb <= '0';
        wait;
    end process;
end architecture test;
''')

In [4]:
%%bash
# Verilog simulation
iverilog -o and_gate_vvp and_gate.v and_gate_tb.v
vvp and_gate_vvp

# VHDL simulation (GHDL)
ghdl -a and_gate.vhd
ghdl -a and_gate_tb.vhd
ghdl -e and_gate_tb
# The simulation below creates a .ghw waveform file
ghdl -r and_gate_tb --wave=and_gate.ghw

VCD info: dumpfile and_gate.vcd opened for output.


In [5]:
%%bash
# Verilog already produces and_gate.vcd
# GHDL produces and_gate.ghw (can be converted to VCD if needed)
# Optional: convert GHDL waveform to VCD for compatibility
if [ -f and_gate.ghw ]; then
  ghdl --convert-wave=and_gate.ghw --output=and_gate_ghdl.vcd
fi
ls -lh *.vcd *.ghw 2>/dev/null || echo "No waveform files generated yet."

-rw-r--r-- 1 root root 414 Jul 20 16:23 and_gate.ghw
-rw-r--r-- 1 root root 405 Jul 20 16:23 and_gate.vcd


/usr/bin/ghdl-llvm: unknown command '--convert-wave=and_gate.ghw', try 'help'


In [6]:
import os
import zipfile
from google.colab import files

# Collect all output and source files
all_files = [
    'and_gate.v', 'and_gate_tb.v', 'and_gate.vhd', 'and_gate_tb.vhd',
    'and_gate.vcd', 'and_gate.ghw', 'and_gate_ghdl.vcd', 'and_gate_vvp'
]

# Only include files that exist
existing_files = [f for f in all_files if os.path.isfile(f)]

zip_filename = 'hdl_outputs.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in existing_files:
        zipf.write(file)

files.download(zip_filename)
print(f"✅ Zipped and downloaded {zip_filename}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Zipped and downloaded hdl_outputs.zip


✅ **All tools are installed and basic HDL simulation completed!**

**Tips for GitHub:**
- You can upload your HDL source files and waveform outputs to your GitHub repository for version control and sharing.
- Use descriptive commit messages and organize your code by language (e.g., place Verilog and VHDL in separate folders).
- Include a `README.md` explaining your project, example commands, and how to run simulations.

Happy hardware design! 🚀